<a href="https://colab.research.google.com/github/burraco135/Notebooks/blob/master/audio_course_automatic_speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic Speech Recognition (ASR)

- Fine-tune the ”openai/whisper-tiny” model using the American English (“en-US”) subset of the ”PolyAI/minds14” dataset.
- Use the first 450 examples for training, and the rest for evaluation. Ensure you set num_proc=1 when pre-processing the dataset using the .map method (this will ensure your model is submitted correctly for assessment).
- To evaluate the model, use the wer and wer_ortho metrics as described in this Unit. However, do not convert the metric into percentages by multiplying by 100 (E.g. if WER is 42%, we’ll expect to see the value of 0.42 in this exercise)

In [ ]:
%pip install -q datasets transformers torch evaluate jiwer
%pip install accelerate -U

In [2]:
from datasets import load_dataset

# load PolyAI/minds14 dataset en-US
minds = load_dataset("PolyAI/minds14", "en-US")

In [3]:
# split train and test sets
minds = minds["train"].train_test_split(seed=42, shuffle=True, test_size=0.2)

In [4]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 450
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 113
    })
})

In [5]:
# remove metadata
minds = minds.select_columns(["audio", "transcription"])

In [6]:
# load processor from checkpoint
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-tiny", language="en", task="transcribe"
)

In [7]:
# preprocess data
minds["train"].features

{'audio': Audio(sampling_rate=8000, mono=True, decode=True, id=None),
 'transcription': Value(dtype='string', id=None)}

In [8]:
# downsample audio
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
minds = minds.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [9]:
# prepare data for the model
def prepare_dataset(example):
  audio = example["audio"]

  example = processor(
      audio=audio["array"],
      sampling_rate=audio["sampling_rate"],
      text=example["transcription"]
  )

  # compute input length of audio sample in seconds
  example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

  return example

In [10]:
# removing columns from the raw training data
minds = minds.map(
    prepare_dataset, remove_columns=minds.column_names["train"], num_proc=1
)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

In [11]:
# filter training data with audio samples longer than 30s
max_input_length = 30.0

def is_audio_in_length_range(length):
  return length < max_input_length

In [12]:
# apply filter
minds["train"] = minds["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"]
)

Filter:   0%|          | 0/450 [00:00<?, ? examples/s]

In [13]:
minds["train"]

Dataset({
    features: ['input_features', 'labels', 'input_length'],
    num_rows: 445
})

In [14]:
# the data collator treats the input_features and label independently
#   input_features must be handled by the feature extractor (already padded)
#   and labels by the tokenizer (unpadded)
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [15]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [16]:
# defining a function that return the WER metric from model predictions
import evaluate

metric = evaluate.load("wer")

In [17]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [18]:
# load a pre-trained checkpoint
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [19]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="english", task="transcribe", use_cache=True
)

In [20]:
# define training configuration
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-en",  # name on the HF Hub
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=500,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [21]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=minds["train"],
    eval_dataset=minds["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

In [22]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Wer Ortho,Wer
500,0.000700,0.709758,0.357335,0.356589


TrainOutput(global_step=500, training_loss=0.34740809054672717, metrics={'train_runtime': 1355.9849, 'train_samples_per_second': 5.9, 'train_steps_per_second': 0.369, 'total_flos': 1.9569551781888e+17, 'train_loss': 0.34740809054672717, 'epoch': 17.86})

In [37]:
# linking the notebook to Hub
from huggingface_hub import notebook_login

notebook_login()

In [38]:
# push the model on the hub
kwargs = {
    "dataset_tags": "PolyAI/minds14",
    "finetuned_from": "openai/whisper-tiny",
    "tasks": "automatic-speech-recognition",
}

trainer.push_to_hub(**kwargs)

Cloning https://huggingface.co/burraco135/whisper-tiny-en into local empty directory.


Upload file pytorch_model.bin:   0%|          | 1.00/144M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/4.00k [00:00<?, ?B/s]

To https://huggingface.co/burraco135/whisper-tiny-en
   3641ab4..05b32a9  main -> main

   3641ab4..05b32a9  main -> main

To https://huggingface.co/burraco135/whisper-tiny-en
   05b32a9..eea732a  main -> main

   05b32a9..eea732a  main -> main



'https://huggingface.co/burraco135/whisper-tiny-en/commit/05b32a9d2e9737d45a5acae9d455a8e3c8918f9d'